# COGS 108 - Final Project 

# Overview

*Fill in your overview here*

### Name & PID

- Name: Carlos Eduardo Matos Ribeiro
- PID: A14032489

# Research Question

Does the location of a restaurant in North Carolina impact its overall sanitary conditions? Is such impact caused by the socioeconomic conditions of a restaurant's neighborhood?

Possible ideas:
- Investigate the connection between neighborhood income and average restaruant "cleaniness"?
- Check number of inspections per zip code and check factors impacting number of inspections (average score of restaurants in the same area, average income of related zip-code)
- Investigate factors that will impact the score of a restaurant (average income of zip-code, result of previous inspections, result of neighboring (how to define? latitude and logitude?) restaurants)


## Background and Prior Work

In North Carolina, restaurants and other food establishments are inspected by local county professionals between one and four times annually$^{1}$. In these inspections, violations are counted and tabulated, each with its own point value depending on its severity; the points are then subtracted from 100 to give a restaurant's final score, which is then used to determine its letter grade$^{1}$. This simple system allows for a straightforward way to quantify the sanitary conditions of restaurants, making it possible to carry out data analyses to investigate general trends and determining factors of said conditions.

With that in mind, I have decided to investgiate whether the location of a restaurant in North Carolina, and more specifically said location's socio-economic condition, has a considerable impact in said restaurant's sanitary conditions. Certain characteristics about neighborhoods, including how busy it is and the average spending potential of its residents/bypassers, can greatly impact key factors for a restaurant's success, including the amount of customers it can expect to have and the amount of revenue that it can generate. Customer traffic and revenue in turn can affect many factors related to sanitary conditions, including: how much staff a restaurant can hire (and how overworked said staff is), how well trained the staff is, the quality of the equipment, the quality of the produce, how often produce has to be restocked, how  much attention managers can devote to health and safety practices, etc. Therefore, it is reasonable to expect that the socio-economic conditions of a restaurant's area will somehow affect its health and safety conditions.

The most relevant research related to my investigation was a study conducted by the Environmental Health Specialists Network, based on several US states, which aimed to identify which factors most impacted safe food preparation practices in a restaurant. The study concluded that some of the main factors were the time pressure the employees felt, the quality of their equipment, the resources they had available and the emphasis managers gave to food safety$^{2}$. As discussed in the previous paragraph, many of these factors can be impacted by a restaurant's location and its socio-economic conditions, so this study helps corroborate that my research question holds promise and is worth investigating.


References (include links):
- 1) https://www.forsyth.cc/PublicHealth/EnvironmentalHealth/aboutInspections.aspx
- 2)https://www.cdc.gov/nceh/ehs/ehsnet/docs/Factors_Impacting_Food_Workers_Food_Prep_FPT_journal.pdf


Possible sources:
- http://data-wake.opendata.arcgis.com/datasets/food-inspections
- https://www.wral.com/yuck-or-yum-inspection-grades-keep-restaurants-diners-safe/16548733/
- https://www.forsyth.cc/PublicHealth/EnvironmentalHealth/aboutInspections.aspx (official article with some description of inspections)
- https://www.starnewsonline.com/news/20190823/on-restaurant-inspections-new-hanover-fights-uphill-battle (article about county in North Carolina falling behind on inspections 

# Hypothesis


I expect that there will be a postive relationship between North Carolina restaurants' location and the socio-economic characteristics of said location, and their overall sanitary conditions, with restaurants in wealthier neighborhoods having fewer health/safety violations then restaurants in low-income, emptier neighborhoods. The reason for that premise was hinted at in the previous section: restaurants in wealthier neighborhoods are more likely to have a constant influx of customers with sufficient spending power to drive their business, thus have the ability to have an adaquately-sized staff that is properly trained and equipped to maintain appropriate health conditions. Conversely, restaurants in low-income neighborhoods are more likely to have sufficient customers/revenue to adequately train and equip their staf, and it might face financial hardships that would lead it to overlook sanitary issues. 

# Dataset(s)

- Dataset Name: Inspections
- Link to the dataset: data/inspections.csv
- Number of observations: 18466
 
In this dataset, each row is a restaurant inspection and each column holds information either about the specific inspection, or the restaurant being inspected. For each inspection there is a myriad of information, including the restaurant's name, address, zip code, and coordinate location, and the the inspection date, number of critical/non-critical violations, overall score, to name a few.

- Dataset Name: Zip
- Link to the dataset: data/zip.csv
- Number of observations: 38

This dataset gathers general socio-economic indicators for various zip codes in North Carolina. Each row is a specific zip code and each column is a different indicator, such as per capita income and median family income and the percent of families living below the poverty line.

The datasets can be combined via the zip codes, with the Zip dataset being used to identify the socio-economic indicators of the neighborhood of each particular inspection.

# Setup

In [1]:
import pandas as pd

# Data Cleaning

First I loaded and examined the inspections dataset

In [13]:
df_inspections = pd.read_csv('data/inspections.csv')
print(df_inspections.columns)
df_inspections.head()

Index(['hsisid', 'date', 'name', 'address1', 'address2', 'city', 'state',
       'postalcode', 'phonenumber', 'restaurantopendate',
       'days_from_open_date', 'facilitytype', 'x', 'y', 'geocodestatus', 'zip',
       'type', 'description', 'inspectedby', 'inspection_num', 'inspector_id',
       'previous_inspection_date', 'days_since_previous_inspection',
       'previous_inspection_by_same_inspector', 'score', 'num_critical',
       'num_non_critical', 'num_critical_previous',
       'num_non_critical_previous', 'num_critical_mean_previous',
       'num_non_critical_mean_previous', 'avg_neighbor_num_critical',
       'avg_neighbor_num_non_critical', 'top_match', 'second_match',
       'critical'],
      dtype='object')


,hsisid,date,name,address1,address2,city,state,postalcode,phonenumber,restaurantopendate,...,num_non_critical,num_critical_previous,num_non_critical_previous,num_critical_mean_previous,num_non_critical_mean_previous,avg_neighbor_num_critical,avg_neighbor_num_non_critical,top_match,second_match,critical
0,4092013748,2012-09-21T00:00:00Z,Cafe 3000 At Wake Med,3000 New Bern Ave,NaN,raleigh,NC,27610,(919) 350-8047,2002-12-21T00:00:00Z,...,7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
1,4092014046,2012-09-21T00:00:00Z,Overtime Sports Pub,1030-149 N Rogers Ln,NaN,raleigh,NC,27610,(919) 255-9556,2004-05-04T00:00:00Z,...,11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
2,4092015191,2012-09-21T00:00:00Z,TASTE OF CHINA,6209 ROCK QUARRY RD,STE 126,raleigh,NC,27610,(919) 773-2285,2008-08-04T00:00:00Z,...,8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
3,4092016122,2012-09-21T00:00:00Z,Panera Bread #1643,1065 Darrington DR,NaN,cary,NC,27513,NaN,2012-03-28T00:00:00Z,...,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
4,4092021513,2012-09-21T00:00:00Z,WalMart Supercenter #4499-00 Deli/Bakery,841 E Gannon AVE,NaN,zebulon,NC,27597,(919) 269-2221 ext. 304,2008-02-25T00:00:00Z,...,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1


Clearly there are several columns that are not needed for my analysis, some of which have some unnecessary personal information (more on this later in the ethics & privacy section), so the first step was to narrow it down to the relevant columns, including zip, coordinates, overall sanitary score and number of each type (critical/non-crtiical) of violation.

In [16]:
df_inspections = df_inspections[['hsisid','x','y','geocodestatus','zip','score','num_critical','num_non_critical']]

,hsisid,x,y,geocodestatus,zip,score,num_critical,num_non_critical
0,4092013748,-78.587963,35.783914,M,27610,96.0,9,7
1,4092014046,-78.549673,35.791986,M,27610,98.0,0,11
2,4092015191,-78.547960,35.723125,M,27610,97.0,4,8
3,4092016122,-78.815727,35.799249,M,27513,99.0,2,3
4,4092021513,-78.296347,35.831285,M,27597,97.0,2,4


Next I ensured that, within the relevant columns, there was no null data to worry about:

In [19]:
pd.isna(df_inspections).any().any()


False

Then I loaded and inspected the zip dataset

In [26]:
df_zipcodes = pd.read_csv('data/zipcodes.csv')
print(df_zipcodes.columns)
df_zipcodes.head()

Index(['zip', 'median_family_income_dollars',
       'median_household_income_dollars', 'per_capita_income_dollars',
       'percent_damilies_below_poverty_line', 'percent_snap_benefits',
       'percent_supplemental_security_income', 'percent_nonwhite'],
      dtype='object')


,zip,median_family_income_dollars,median_household_income_dollars,per_capita_income_dollars,percent_damilies_below_poverty_line,percent_snap_benefits,percent_supplemental_security_income,percent_nonwhite
0,27501,59408,51121,21631,10.5,15.5,5.2,17.9
1,27502,109891,95857,36763,3.4,2.4,0.8,18.9
2,27511,82292,67392,33139,9.6,4.5,2.2,24.8
3,27513,109736,87262,41232,3.8,2.4,1.5,27.8
4,27518,125432,98247,49865,5.5,1.0,1.7,19.9


Next, I renamed a column to fix a typo, again dropped some of the columns that had information that was either unncessary, redundant and/or problematic (more on the ethics and privacy section), and checked that there was no null data to worry about.

In [28]:
df_zipcodes = df_zipcodes.rename(columns={"percent_damilies_below_poverty_line": "percent_families_below_poverty_line"})
df_zipcodes = df_zipcodes[['zip','median_household_income_dollars','per_capita_income_dollars','percent_families_below_poverty_line']]
pd.isna(df_zipcodes).any().any()

False

With both datasets in structured format, streamlined to keep only potentially relevant informatoin, and free of null values, they were both ready for analysis and no further cleaning was required.

# Data Analysis & Results

Include cells that describe the steps in your data analysis.

In [5]:
## YOUR CODE HERE
## FEEL FREE TO ADD MULTIPLE CELLS PER SECTION

# Ethics & Privacy

*Fill in your ethics & privacy discussion here*

# Conclusion & Discussion

*Fill in your discussion information here*